In [1]:
import pandas as pd
import glob
from DeepMRI import DeepMRI
import SegAN_arch as segan_original
import SegAN_IO_arch as seganio
import seaborn as sb
import matplotlib.pyplot as plt

models_path = 'models/'

sb.set_style("whitegrid")
sb.set_context("paper", font_scale=2)

brats2019 = {
           'training':'brats2019_training_crop_mri',
           'validation':'brats2019_validation_crop_mri',
           'testing':'brats2019_testing_crop_mri'
            }

brats2015 = {
           'training':'brats2015_training_crop_mri',
           'validation':'brats2015_validation_crop_mri',
           'testing':'brats2015_testing_crop_mri'
            }



## Print best models (validation) for each run in './models/'

In [2]:
def run_evaluation(model_name, model_epoch, mri_types, architecture, dataset, dataset_split='both', tracked_metric='dice_score', output_labels=1, model_input_size=160, seed=1234567890):
    assert dataset_split in ['testing', 'validation', 'both', 'merged']
    model_checkpoint = glob.glob(models_path + model_name + '/' + 'best_' +  tracked_metric + '_' + str(model_epoch) + '*.index')[0].replace('.index', '')
    gan = DeepMRI(batch_size=64, size=model_input_size, mri_channels=len(mri_types), output_labels=output_labels, model_name=model_name)
    gan.load_dataset(dataset, mri_types)
    gan.build_model(load_model=model_checkpoint, seed=seed, arch=architecture)
    stat_metrics = ['true_positives', 'false_positives', 'false_negatives', 'true_negatives']
    performances = pd.DataFrame()
    if dataset_split in ['both', 'validation']:
        valid_scores = gan.evaluate(dataset='validation')
        valid_scores['run'] = 'validation'
        valid_scores['model'] = model_name
        performances = performances.append(valid_scores, ignore_index=True)
    if dataset_split in ['both', 'testing']: 
        test_scores = gan.evaluate(dataset='testing')
        test_scores['run'] = 'testing'
        test_scores['model'] = model_name
        performances = performances.append(test_scores, ignore_index=True)
    if dataset_split == 'merged':
        merged_scores = gan.evaluate(dataset='merged')
        merged_scores['run'] = 'merged'
        merged_scores['model'] = model_name
        performances = performances.append(merged_scores, ignore_index=True)
    return performances.set_index(['model', 'run'])
    


In [ ]:
metric='dice_score'
for csv in glob.glob('models/*/log_valid.csv'):
    modelname = csv.split('/')[1]
    pdcsv = pd.read_csv(csv)
    best = pdcsv.loc[pdcsv['dice_score'].idxmax()]
    print("Model: {} - Best {} {} at epoch {}".format(modelname, metric, best['dice_score'], best['epoch']))

# Base Models (All Modalities)

In [ ]:
base_paper = run_evaluation("Segan_TF2_brats_ALL", model_epoch=1386, mri_types=["MR_T1", "MR_T1c", "MR_T2", "MR_Flair"], architecture=segan_original, dataset=brats2015, dataset_split='merged')
base_nodice = run_evaluation("Segan_NoDice_TF2_brats_ALL", model_epoch=842, mri_types=["MR_T1", "MR_T1c", "MR_T2", "MR_Flair"], architecture=segan_original, dataset=brats2015, dataset_split='merged')
our_full = run_evaluation("Segan_IO_TF2_brats_ALL", model_epoch=1123, mri_types=["MR_T1", "MR_T1c", "MR_T2", "MR_Flair"], architecture=seganio, dataset=brats2015, dataset_split='merged')
basemodels = pd.concat([base_paper, base_nodice, our_full])

In [3]:
base_nodice_2019 = run_evaluation("Segan_NoDice_TF2_brats2019_ALL", model_epoch=138, mri_types=["t1", "t1ce", "t2", "flair"], architecture=segan_original, dataset=brats2019, tracked_metric="dice_score_0", dataset_split='validation')
base_nodice_2019

Loading training dataset brats2019_training_crop_mri with modalities t1,t1ce,t2,flair
Loading validation dataset brats2019_validation_crop_mri with modalities t1,t1ce,t2,flair
Loading testing dataset brats2019_testing_crop_mri with modalities t1,t1ce,t2,flair
Done.
Using architecture: SegAN_arch
Loading models/Segan_NoDice_TF2_brats2019_ALL/best_dice_score_0_138-24
Resuming model from: models/Segan_NoDice_TF2_brats2019_ALL/best_dice_score_0_138-24, next epoch: 139
     81/Unknown - 26s 318ms/step - loss_g: 0.8072 - loss_d: -0.1131

,,dice_score_0,precision_0,sensitivity_0,loss_g,loss_d,learning_rate_g,learning_rate_d,dice_score_complete_tumor_2019,precision_complete_tumor_2019,sensitivity_complete_tumor_2019,...,dice_score_complete_tumor,precision_complete_tumor,sensitivity_complete_tumor,dice_score_tumor_core,precision_tumor_core,sensitivity_tumor_core,dice_score_enhancing_tumor,precision_enhancing_tumor,sensitivity_enhancing_tumor,epoch
model,run,,,,,,,,,,,,,,,,,,,,,
Segan_NoDice_TF2_brats2019_ALL,validation,0.766374,0.745271,0.833709,0.805934,-0.114401,0.00002,0.00002,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


# Base Models (Single Modality)

In [ ]:
flair = run_evaluation("Segan_IO_TF2_brats_on_FLAIR", model_epoch=168, mri_types=["MR_Flair"], architecture=seganio, dataset=brats2015, dataset_split='merged')
t1 = run_evaluation("Segan_IO_TF2_brats_on_T1", model_epoch=861, mri_types=["MR_T1"], architecture=seganio, dataset=brats2015, dataset_split='merged')
t1c = run_evaluation("Segan_IO_TF2_brats_on_T1c", model_epoch=1011, mri_types=["MR_T1c"], architecture=seganio, dataset=brats2015, dataset_split='merged')
t2 = run_evaluation("Segan_IO_TF2_brats_on_T2", model_epoch=182, mri_types=["MR_T2"], architecture=seganio, dataset=brats2015, dataset_split='merged')
singlemod = pd.concat([t1, t1c, t2, flair])

# Performances of Base Models on Other Modalities

In [11]:
## 2015

In [ ]:
# T1
t1_on_t1 = run_evaluation("Segan_IO_TF2_brats_on_T1", model_epoch=861, mri_types=["MR_T1"], architecture=seganio, dataset=brats2015, dataset_split='merged')
t1_on_t1['Source Modality'] = "T1"
t1_on_t1['Target Modality'] = "T1"

t1_on_t1c = run_evaluation("Segan_IO_TF2_brats_on_T1", model_epoch=861, mri_types=["MR_T1c"], architecture=seganio, dataset=brats2015, dataset_split='merged')
t1_on_t1c['Source Modality'] = "T1"
t1_on_t1c['Target Modality'] = "T1c"

t1_on_t2 = run_evaluation("Segan_IO_TF2_brats_on_T1", model_epoch=861, mri_types=["MR_T2"], architecture=seganio, dataset=brats2015, dataset_split='merged')
t1_on_t2['Source Modality'] = "T1"
t1_on_t2['Target Modality'] = "T2"

t1_on_Flair = run_evaluation("Segan_IO_TF2_brats_on_T1", model_epoch=861, mri_types=["MR_Flair"], architecture=seganio, dataset=brats2015, dataset_split='merged')
t1_on_Flair['Source Modality'] = "T1"
t1_on_Flair['Target Modality'] = "Flair"

# T1c
t1c_on_t1 = run_evaluation("Segan_IO_TF2_brats_on_T1c", model_epoch=1011, mri_types=["MR_T1"], architecture=seganio, dataset=brats2015, dataset_split='merged')
t1c_on_t1['Source Modality'] = "T1c"
t1c_on_t1['Target Modality'] = "T1"
t1c_on_t1c = run_evaluation("Segan_IO_TF2_brats_on_T1c", model_epoch=1011, mri_types=["MR_T1c"], architecture=seganio, dataset=brats2015, dataset_split='merged')
t1c_on_t1c['Source Modality'] = "T1c"
t1c_on_t1c['Target Modality'] = "T1c"
t1c_on_t2 = run_evaluation("Segan_IO_TF2_brats_on_T1c", model_epoch=1011, mri_types=["MR_T2"], architecture=seganio, dataset=brats2015, dataset_split='merged')
t1c_on_t2['Source Modality'] = "T1c"
t1c_on_t2['Target Modality'] = "T2"
t1c_on_Flair = run_evaluation("Segan_IO_TF2_brats_on_T1c", model_epoch=1011, mri_types=["MR_Flair"], architecture=seganio, dataset=brats2015, dataset_split='merged')
t1c_on_Flair['Source Modality'] = "T1c"
t1c_on_Flair['Target Modality'] = "Flair"

# T2
t2_on_t1 = run_evaluation("Segan_IO_TF2_brats_on_T2", model_epoch=182, mri_types=["MR_T1"], architecture=seganio, dataset=brats2015, dataset_split='merged')
t2_on_t1['Source Modality'] = "T2"
t2_on_t1['Target Modality'] = "T1"
t2_on_t1c = run_evaluation("Segan_IO_TF2_brats_on_T2", model_epoch=182, mri_types=["MR_T1c"], architecture=seganio, dataset=brats2015, dataset_split='merged')
t2_on_t1c['Source Modality'] = "T2"
t2_on_t1c['Target Modality'] = "T1c"
t2_on_t2 = run_evaluation("Segan_IO_TF2_brats_on_T2", model_epoch=182, mri_types=["MR_T2"], architecture=seganio, dataset=brats2015, dataset_split='merged')
t2_on_t2['Source Modality'] = "T2"
t2_on_t2['Target Modality'] = "T2"
t2_on_Flair = run_evaluation("Segan_IO_TF2_brats_on_T2", model_epoch=182, mri_types=["MR_Flair"], architecture=seganio, dataset=brats2015, dataset_split='merged')
t2_on_Flair['Source Modality'] = "T2"
t2_on_Flair['Target Modality'] = "Flair"

# FLAIR
flair_on_t1 = run_evaluation("Segan_IO_TF2_brats_on_FLAIR", model_epoch=168, mri_types=["MR_T1"], architecture=seganio, dataset=brats2015, dataset_split='merged')
flair_on_t1['Source Modality'] = "Flair"
flair_on_t1['Target Modality'] = "T1"
flair_on_t1c = run_evaluation("Segan_IO_TF2_brats_on_FLAIR", model_epoch=168, mri_types=["MR_T1c"], architecture=seganio, dataset=brats2015, dataset_split='merged')
flair_on_t1c['Source Modality'] = "Flair"
flair_on_t1c['Target Modality'] = "T1c"
flair_on_t2 = run_evaluation("Segan_IO_TF2_brats_on_FLAIR", model_epoch=168, mri_types=["MR_T2"], architecture=seganio, dataset=brats2015, dataset_split='merged')
flair_on_t2['Source Modality'] = "Flair"
flair_on_t2['Target Modality'] = "T2"
flair_on_Flair = run_evaluation("Segan_IO_TF2_brats_on_FLAIR", model_epoch=168, mri_types=["MR_Flair"], architecture=seganio, dataset=brats2015, dataset_split='merged')
flair_on_Flair['Source Modality'] = "Flair"
flair_on_Flair['Target Modality'] = "Flair"

In [10]:
base_performances = pd.DataFrame()
base_performances = base_performances.append(t1_on_t1.reset_index(), ignore_index=True)
base_performances = base_performances.append(t1_on_t1c.reset_index(), ignore_index=True)
base_performances = base_performances.append(t1_on_t2.reset_index(), ignore_index=True)
base_performances = base_performances.append(t1_on_Flair.reset_index(), ignore_index=True)
base_performances = base_performances.append(t1c_on_t1.reset_index(), ignore_index=True)
base_performances = base_performances.append(t1c_on_t1c.reset_index(), ignore_index=True)
base_performances = base_performances.append(t1c_on_t2.reset_index(), ignore_index=True)
base_performances = base_performances.append(t1c_on_Flair.reset_index(), ignore_index=True)
base_performances = base_performances.append(t2_on_t1.reset_index(), ignore_index=True)
base_performances = base_performances.append(t2_on_t1c.reset_index(), ignore_index=True)
base_performances = base_performances.append(t2_on_t2.reset_index(), ignore_index=True)
base_performances = base_performances.append(t2_on_Flair.reset_index(), ignore_index=True)
base_performances = base_performances.append(flair_on_t1.reset_index(), ignore_index=True)
base_performances = base_performances.append(flair_on_t1c.reset_index(), ignore_index=True)
base_performances = base_performances.append(flair_on_t2.reset_index(), ignore_index=True)
base_performances = base_performances.append(flair_on_Flair.reset_index(), ignore_index=True)

base_performances.to_csv('results/base_models_on_other_modalities_2015.csv')

In [3]:
## 2019

# T1
t1_on_t1 = run_evaluation("Segan_IO_TF2_brats2019_T1", model_epoch=124, mri_types=["t1"], architecture=seganio, dataset=brats2019, tracked_metric='dice_score_0', dataset_split='validation')
t1_on_t1['Source Modality'] = "T1"
t1_on_t1['Target Modality'] = "T1"

t1_on_t1c = run_evaluation("Segan_IO_TF2_brats2019_T1", model_epoch=124, mri_types=["t1ce"], architecture=seganio, dataset=brats2019, tracked_metric='dice_score_0', dataset_split='validation')
t1_on_t1c['Source Modality'] = "T1"
t1_on_t1c['Target Modality'] = "T1c"

t1_on_t2 = run_evaluation("Segan_IO_TF2_brats2019_T1", model_epoch=124, mri_types=["t2"], architecture=seganio, dataset=brats2019, tracked_metric='dice_score_0', dataset_split='validation')
t1_on_t2['Source Modality'] = "T1"
t1_on_t2['Target Modality'] = "T2"

t1_on_Flair = run_evaluation("Segan_IO_TF2_brats2019_T1", model_epoch=124, mri_types=["flair"], architecture=seganio, dataset=brats2019, tracked_metric='dice_score_0', dataset_split='validation')
t1_on_Flair['Source Modality'] = "T1"
t1_on_Flair['Target Modality'] = "Flair"

# T1c
t1c_on_t1 = run_evaluation("Segan_IO_TF2_brats2019_T1ce", model_epoch=597, mri_types=["t1"], architecture=seganio, dataset=brats2019, tracked_metric='dice_score_0', dataset_split='validation')
t1c_on_t1['Source Modality'] = "T1c"
t1c_on_t1['Target Modality'] = "T1"
t1c_on_t1c = run_evaluation("Segan_IO_TF2_brats2019_T1ce", model_epoch=597, mri_types=["t1ce"], architecture=seganio, dataset=brats2019, tracked_metric='dice_score_0', dataset_split='validation')
t1c_on_t1c['Source Modality'] = "T1c"
t1c_on_t1c['Target Modality'] = "T1c"
t1c_on_t2 = run_evaluation("Segan_IO_TF2_brats2019_T1ce", model_epoch=597, mri_types=["t2"], architecture=seganio, dataset=brats2019, tracked_metric='dice_score_0', dataset_split='validation')
t1c_on_t2['Source Modality'] = "T1c"
t1c_on_t2['Target Modality'] = "T2"
t1c_on_Flair = run_evaluation("Segan_IO_TF2_brats2019_T1ce", model_epoch=597, mri_types=["flair"], architecture=seganio, dataset=brats2019, tracked_metric='dice_score_0', dataset_split='validation')
t1c_on_Flair['Source Modality'] = "T1c"
t1c_on_Flair['Target Modality'] = "Flair"

# T2
t2_on_t1 = run_evaluation("Segan_IO_TF2_brats2019_T2", model_epoch=136, mri_types=["t1"], architecture=seganio, dataset=brats2019, tracked_metric='dice_score_0', dataset_split='validation')
t2_on_t1['Source Modality'] = "T2"
t2_on_t1['Target Modality'] = "T1"
t2_on_t1c = run_evaluation("Segan_IO_TF2_brats2019_T2", model_epoch=136, mri_types=["t1ce"], architecture=seganio, dataset=brats2019, tracked_metric='dice_score_0', dataset_split='validation')
t2_on_t1c['Source Modality'] = "T2"
t2_on_t1c['Target Modality'] = "T1c"
t2_on_t2 = run_evaluation("Segan_IO_TF2_brats2019_T2", model_epoch=136, mri_types=["t2"], architecture=seganio, dataset=brats2019, tracked_metric='dice_score_0', dataset_split='validation')
t2_on_t2['Source Modality'] = "T2"
t2_on_t2['Target Modality'] = "T2"
t2_on_Flair = run_evaluation("Segan_IO_TF2_brats2019_T2", model_epoch=136, mri_types=["flair"], architecture=seganio, dataset=brats2019, tracked_metric='dice_score_0', dataset_split='validation')
t2_on_Flair['Source Modality'] = "T2"
t2_on_Flair['Target Modality'] = "Flair"

# FLAIR
flair_on_t1 = run_evaluation("Segan_IO_TF2_brats2019_FLAIR", model_epoch=89, mri_types=["t1"], architecture=seganio, dataset=brats2019, tracked_metric='dice_score_0', dataset_split='validation')
flair_on_t1['Source Modality'] = "Flair"
flair_on_t1['Target Modality'] = "T1"
flair_on_t1c = run_evaluation("Segan_IO_TF2_brats2019_FLAIR", model_epoch=89, mri_types=["t1ce"], architecture=seganio, dataset=brats2019, tracked_metric='dice_score_0', dataset_split='validation')
flair_on_t1c['Source Modality'] = "Flair"
flair_on_t1c['Target Modality'] = "T1c"
flair_on_t2 = run_evaluation("Segan_IO_TF2_brats2019_FLAIR", model_epoch=89, mri_types=["t2"], architecture=seganio, dataset=brats2019, tracked_metric='dice_score_0', dataset_split='validation')
flair_on_t2['Source Modality'] = "Flair"
flair_on_t2['Target Modality'] = "T2"
flair_on_Flair = run_evaluation("Segan_IO_TF2_brats2019_FLAIR", model_epoch=89, mri_types=["flair"], architecture=seganio, dataset=brats2019, tracked_metric='dice_score_0', dataset_split='validation')
flair_on_Flair['Source Modality'] = "Flair"
flair_on_Flair['Target Modality'] = "Flair"

Loading training dataset brats2019_training_crop_mri with modalities t1
Loading validation dataset brats2019_validation_crop_mri with modalities t1
Loading testing dataset brats2019_testing_crop_mri with modalities t1
Done.
Using architecture: SegAN_IO_arch
Loading models/Segan_IO_TF2_brats2019_T1/best_dice_score_0_124-10
Resuming model from: models/Segan_IO_TF2_brats2019_T1/best_dice_score_0_124-10, next epoch: 125
     81/Unknown - 25s 306ms/step - loss_g: 0.4709 - loss_d: -0.0957Loading training dataset brats2019_training_crop_mri with modalities t1ce
Loading validation dataset brats2019_validation_crop_mri with modalities t1ce
Loading testing dataset brats2019_testing_crop_mri with modalities t1ce
Done.
Using architecture: SegAN_IO_arch


W0113 10:50:56.732568 140552759818048 util.py:244] Unresolved object in checkpoint: (root).d_optimizer.iter
W0113 10:50:56.734035 140552759818048 util.py:252] A checkpoint was restored (e.g. tf.train.Checkpoint.restore or tf.keras.Model.load_weights) but not all checkpointed values were used. See above for specific issues. Use expect_partial() on the load status object, e.g. tf.train.Checkpoint.restore(...).expect_partial(), to silence these warnings, or use assert_consumed() to make the check explicit. See https://www.tensorflow.org/alpha/guide/checkpoints#loading_mechanics for details.


Loading models/Segan_IO_TF2_brats2019_T1/best_dice_score_0_124-10
Resuming model from: models/Segan_IO_TF2_brats2019_T1/best_dice_score_0_124-10, next epoch: 125
     81/Unknown - 21s 255ms/step - loss_g: 0.6990 - loss_d: -0.1385Loading training dataset brats2019_training_crop_mri with modalities t2
Loading validation dataset brats2019_validation_crop_mri with modalities t2
Loading testing dataset brats2019_testing_crop_mri with modalities t2
Done.
Using architecture: SegAN_IO_arch


W0113 10:51:20.077019 140552759818048 util.py:244] Unresolved object in checkpoint: (root).d_optimizer.iter
W0113 10:51:20.078498 140552759818048 util.py:252] A checkpoint was restored (e.g. tf.train.Checkpoint.restore or tf.keras.Model.load_weights) but not all checkpointed values were used. See above for specific issues. Use expect_partial() on the load status object, e.g. tf.train.Checkpoint.restore(...).expect_partial(), to silence these warnings, or use assert_consumed() to make the check explicit. See https://www.tensorflow.org/alpha/guide/checkpoints#loading_mechanics for details.


Loading models/Segan_IO_TF2_brats2019_T1/best_dice_score_0_124-10
Resuming model from: models/Segan_IO_TF2_brats2019_T1/best_dice_score_0_124-10, next epoch: 125
     81/Unknown - 21s 254ms/step - loss_g: 1.3078 - loss_d: -0.4502Loading training dataset brats2019_training_crop_mri with modalities flair
Loading validation dataset brats2019_validation_crop_mri with modalities flair
Loading testing dataset brats2019_testing_crop_mri with modalities flair
Done.
Using architecture: SegAN_IO_arch


...

Done.
Using architecture: SegAN_IO_arch
Loading models/Segan_IO_TF2_brats2019_T2/best_dice_score_0_136-11
Resuming model from: models/Segan_IO_TF2_brats2019_T2/best_dice_score_0_136-11, next epoch: 137
     81/Unknown - 21s 255ms/step - loss_g: 1.2913 - loss_d: -0.4266Loading training dataset brats2019_training_crop_mri with modalities t1ce
Loading validation dataset brats2019_validation_crop_mri with modalities t1ce
Loading testing dataset brats2019_testing_crop_mri with modalities t1ce
Done.
Using architecture: SegAN_IO_arch
Loading models/Segan_IO_TF2_brats2019_T2/best_dice_score_0_136-11
Resuming model from: models/Segan_IO_TF2_brats2019_T2/best_dice_score_0_136-11, next epoch: 137
     81/Unknown - 21s 256ms/step - loss_g: 1.1072 - loss_d: -0.2855Loading training dataset brats2019_training_crop_mri with modalities t2
Loading validation dataset brats2019_validation_crop_mri with modalities t2
Loading testing dataset brats2019_testing_crop_mri with modalities t2
Done.
Using architec

W0113 10:54:27.474389 140552759818048 util.py:244] Unresolved object in checkpoint: (root).d_optimizer.iter
W0113 10:54:27.475749 140552759818048 util.py:252] A checkpoint was restored (e.g. tf.train.Checkpoint.restore or tf.keras.Model.load_weights) but not all checkpointed values were used. See above for specific issues. Use expect_partial() on the load status object, e.g. tf.train.Checkpoint.restore(...).expect_partial(), to silence these warnings, or use assert_consumed() to make the check explicit. See https://www.tensorflow.org/alpha/guide/checkpoints#loading_mechanics for details.
W0113 10:54:27.480953 140552759818048 util.py:244] Unresolved object in checkpoint: (root).d_optimizer.iter
W0113 10:54:27.481745 140552759818048 util.py:252] A checkpoint was restored (e.g. tf.train.Checkpoint.restore or tf.keras.Model.load_weights) but not all checkpointed values were used. See above for specific issues. Use expect_partial() on the load status object, e.g. tf.train.Checkpoint.restore

Loading models/Segan_IO_TF2_brats2019_T2/best_dice_score_0_136-11
Resuming model from: models/Segan_IO_TF2_brats2019_T2/best_dice_score_0_136-11, next epoch: 137
     81/Unknown - 21s 256ms/step - loss_g: 0.5000 - loss_d: -0.0854Loading training dataset brats2019_training_crop_mri with modalities flair
Loading validation dataset brats2019_validation_crop_mri with modalities flair
Loading testing dataset brats2019_testing_crop_mri with modalities flair
Done.
Using architecture: SegAN_IO_arch
Loading models/Segan_IO_TF2_brats2019_T2/best_dice_score_0_136-11
Resuming model from: models/Segan_IO_TF2_brats2019_T2/best_dice_score_0_136-11, next epoch: 137
     81/Unknown - 21s 254ms/step - loss_g: 0.5826 - loss_d: -0.1265Loading training dataset brats2019_training_crop_mri with modalities t1
Loading validation dataset brats2019_validation_crop_mri with modalities t1
Loading testing dataset brats2019_testing_crop_mri with modalities t1
Done.
Using architecture: SegAN_IO_arch
Loading models/Se

W0113 10:55:30.986872 140552759818048 util.py:244] Unresolved object in checkpoint: (root).d_optimizer.iter
W0113 10:55:30.988150 140552759818048 util.py:252] A checkpoint was restored (e.g. tf.train.Checkpoint.restore or tf.keras.Model.load_weights) but not all checkpointed values were used. See above for specific issues. Use expect_partial() on the load status object, e.g. tf.train.Checkpoint.restore(...).expect_partial(), to silence these warnings, or use assert_consumed() to make the check explicit. See https://www.tensorflow.org/alpha/guide/checkpoints#loading_mechanics for details.
W0113 10:55:30.993757 140552759818048 util.py:244] Unresolved object in checkpoint: (root).d_optimizer.iter
W0113 10:55:30.994690 140552759818048 util.py:252] A checkpoint was restored (e.g. tf.train.Checkpoint.restore or tf.keras.Model.load_weights) but not all checkpointed values were used. See above for specific issues. Use expect_partial() on the load status object, e.g. tf.train.Checkpoint.restore

     81/Unknown - 21s 256ms/step - loss_g: 1.0288 - loss_d: -0.3057Loading training dataset brats2019_training_crop_mri with modalities t1ce
Loading validation dataset brats2019_validation_crop_mri with modalities t1ce
Loading testing dataset brats2019_testing_crop_mri with modalities t1ce
Done.
Using architecture: SegAN_IO_arch
Loading models/Segan_IO_TF2_brats2019_FLAIR/best_dice_score_0_89-10
Resuming model from: models/Segan_IO_TF2_brats2019_FLAIR/best_dice_score_0_89-10, next epoch: 90
     81/Unknown - 21s 254ms/step - loss_g: 0.7938 - loss_d: -0.1585Loading training dataset brats2019_training_crop_mri with modalities t2
Loading validation dataset brats2019_validation_crop_mri with modalities t2
Loading testing dataset brats2019_testing_crop_mri with modalities t2
Done.
Using architecture: SegAN_IO_arch
Loading models/Segan_IO_TF2_brats2019_FLAIR/best_dice_score_0_89-10
Resuming model from: models/Segan_IO_TF2_brats2019_FLAIR/best_dice_score_0_89-10, next epoch: 90
     81/Unknow

In [4]:
base_performances = pd.DataFrame()
base_performances = base_performances.append(t1_on_t1.reset_index(), ignore_index=True)
base_performances = base_performances.append(t1_on_t1c.reset_index(), ignore_index=True)
base_performances = base_performances.append(t1_on_t2.reset_index(), ignore_index=True)
base_performances = base_performances.append(t1_on_Flair.reset_index(), ignore_index=True)
base_performances = base_performances.append(t1c_on_t1.reset_index(), ignore_index=True)
base_performances = base_performances.append(t1c_on_t1c.reset_index(), ignore_index=True)
base_performances = base_performances.append(t1c_on_t2.reset_index(), ignore_index=True)
base_performances = base_performances.append(t1c_on_Flair.reset_index(), ignore_index=True)
base_performances = base_performances.append(t2_on_t1.reset_index(), ignore_index=True)
base_performances = base_performances.append(t2_on_t1c.reset_index(), ignore_index=True)
base_performances = base_performances.append(t2_on_t2.reset_index(), ignore_index=True)
base_performances = base_performances.append(t2_on_Flair.reset_index(), ignore_index=True)
base_performances = base_performances.append(flair_on_t1.reset_index(), ignore_index=True)
base_performances = base_performances.append(flair_on_t1c.reset_index(), ignore_index=True)
base_performances = base_performances.append(flair_on_t2.reset_index(), ignore_index=True)
base_performances = base_performances.append(flair_on_Flair.reset_index(), ignore_index=True)

base_performances.to_csv('results/base_models_on_other_modalities_2019.csv')

# Transfer from FLAIR

In [ ]:
transfer_t1 = run_evaluation("Transfer_Brats_Flair_to_T1", model_epoch=273, mri_types=["MR_T1"], architecture=seganio, dataset=brats2015, dataset_split='merged')
transfer_t1_freeze = run_evaluation("Transfer_Brats_Flair_to_T1_freeze_all", model_epoch=1122, mri_types=["MR_T1"], architecture=seganio, dataset=brats2015, dataset_split='merged')

transfer_t1c = run_evaluation("Transfer_Brats_Flair_to_T1c", model_epoch=722, mri_types=["MR_T1c"], architecture=seganio, dataset=brats2015, dataset_split='merged')
transfer_t1c_freeze = run_evaluation("Transfer_Brats_Flair_to_T1c_freeze_all", model_epoch=751, mri_types=["MR_T1c"], architecture=seganio, dataset=brats2015, dataset_split='merged')

transfer_t2 = run_evaluation("Transfer_Brats_Flair_to_T2", model_epoch=1522, mri_types=["MR_T2"], architecture=seganio, dataset=brats2015, dataset_split='merged')
transfer_t2_freeze = run_evaluation("Transfer_Brats_Flair_to_T2_freeze_all", model_epoch=649, mri_types=["MR_T2"], architecture=seganio, dataset=brats2015, dataset_split='merged')

transfer = pd.concat([transfer_t1, transfer_t1_freeze, transfer_t1c, transfer_t1c_freeze, transfer_t2, transfer_t2_freeze])

## Decorating and saving CSV

In [ ]:
full_results = pd.concat([basemodels, singlemod, transfer]).reset_index()
# Extracting information from model names
def fine_tuning(x):
    if 'transfer' not in x.lower():
        return 'From Scratch'
    else:
        if 'freeze' in x.lower():
            return 'Generator'
        else:
            return 'Full'

def model_type(x):
    if 'transfer' not in x.lower():
        return 'Base Model'
    else:
        return "Transfer"
        

def source(x):
    modelname = x.replace('-', '_') # Fix inconsistencies in model names
    splitted = modelname.split('_')
    if 'transfer' not in modelname.lower():
        source = splitted[-1]
    else:
        to_index = splitted.index('to')
        source = splitted[to_index-1]
    return source

def target(x):
    modelname = x.replace('-', '_') # Fix inconsistencies in model names
    splitted = modelname.split('_')
    if 'transfer' not in modelname.lower():
        dest = splitted[-1]
    else:
        to_index = splitted.index('to')
        dest = splitted[to_index+1]
    return dest
        
full_results['Model Type'] = full_results['model'].apply(model_type)
full_results['Fine Tuning'] = full_results['model'].apply(fine_tuning)
full_results['Source Modality'] = full_results['model'].apply(source)
full_results['Target Modality'] = full_results['model'].apply(target)



In [6]:
full_results.to_csv('results/model_performances_brats2015_mergedsplits.csv', index=False, index_label=False)

NameError: name 'full_results' is not defined

# Evaluating models on other datasets

In [ ]:
## 2019
# Flair (no transfer) on other modalities
flair2019_on_t1_2019 = run_evaluation("Segan_IO_TF2_brats2019_FLAIR", model_epoch=89, mri_types=["t1"], tracked_metric='dice_score_0', architecture=seganio, dataset=brats2019, dataset_split='validation')
flair2019_on_t1ce_2019 = run_evaluation("Segan_IO_TF2_brats2019_FLAIR", model_epoch=89, mri_types=["t1ce"], tracked_metric='dice_score_0', architecture=seganio, dataset=brats2019, dataset_split='validation')
flair2019_on_t2_2019 = run_evaluation("Segan_IO_TF2_brats2019_FLAIR", model_epoch=89, mri_types=["t2"], tracked_metric='dice_score_0', architecture=seganio, dataset=brats2019, dataset_split='validation')


In [ ]:
flair2015_on_t1_2015 = run_evaluation("Segan_IO_TF2_brats_on_FLAIR", model_epoch=168, mri_types=["MR_T1"], tracked_metric='dice_score', architecture=seganio, dataset=brats2015, dataset_split='merged')
flair2015_on_t1ce_2015 = run_evaluation("Segan_IO_TF2_brats_on_FLAIR", model_epoch=168, mri_types=["MR_T1c"], tracked_metric='dice_score', architecture=seganio, dataset=brats2015, dataset_split='merged')
flair2015_on_t2_2015 = run_evaluation("Segan_IO_TF2_brats_on_FLAIR", model_epoch=168, mri_types=["MR_T2"], tracked_metric='dice_score', architecture=seganio, dataset=brats2015, dataset_split='merged')

In [ ]:
flair2015_on_t1_2019 = run_evaluation("Segan_IO_TF2_brats_on_FLAIR", model_epoch=168, mri_types=["t1"], tracked_metric='dice_score', architecture=seganio, dataset=brats2019, dataset_split='validation')
flair2015_on_t1ce_2019 = run_evaluation("Segan_IO_TF2_brats_on_FLAIR", model_epoch=168, mri_types=["t1ce"], tracked_metric='dice_score', architecture=seganio, dataset=brats2019, dataset_split='validation')
flair2015_on_t2_2019 = run_evaluation("Segan_IO_TF2_brats_on_FLAIR", model_epoch=168, mri_types=["t2"], tracked_metric='dice_score', architecture=seganio, dataset=brats2019, dataset_split='validation')

In [ ]:
flair2019_on_t1_2015 = run_evaluation("Segan_IO_TF2_brats2019_FLAIR", model_epoch=89, mri_types=["MR_T1"], tracked_metric='dice_score_0', architecture=seganio, dataset=brats2015, dataset_split='merged')
flair2019_on_t1ce_2015 = run_evaluation("Segan_IO_TF2_brats2019_FLAIR", model_epoch=89, mri_types=["MR_T1c"], tracked_metric='dice_score_0', architecture=seganio, dataset=brats2015, dataset_split='merged')
flair2019_on_t2_2015 = run_evaluation("Segan_IO_TF2_brats2019_FLAIR", model_epoch=89, mri_types=["MR_T2"], tracked_metric='dice_score_0', architecture=seganio, dataset=brats2015, dataset_split='merged')

### Forgetting past samples

In [ ]:
transfer2015_t1_on_flair2015 = run_evaluation("Transfer_Brats_Flair_to_T1", model_epoch=273, mri_types=["MR_Flair"], architecture=seganio, dataset=brats2015, dataset_split='merged')
transfer2015_t1_freeze_on_flair2015 = run_evaluation("Transfer_Brats_Flair_to_T1_freeze_all", model_epoch=1122, mri_types=["MR_Flair"], architecture=seganio, dataset=brats2015, dataset_split='merged')

transfer2015_t1_on_flair2015c = run_evaluation("Transfer_Brats_Flair_to_T1c", model_epoch=722, mri_types=["MR_Flair"], architecture=seganio, dataset=brats2015, dataset_split='merged')
transfer2015_t1c_freeze_on_flair2015 = run_evaluation("Transfer_Brats_Flair_to_T1c_freeze_all", model_epoch=751, mri_types=["MR_Flair"], architecture=seganio, dataset=brats2015, dataset_split='merged')

transfer2015_t2_on_flair2015 = run_evaluation("Transfer_Brats_Flair_to_T2", model_epoch=1522, mri_types=["MR_Flair"], architecture=seganio, dataset=brats2015, dataset_split='merged')
transfer2015_t2_freeze_on_flair2015 = run_evaluation("Transfer_Brats_Flair_to_T2_freeze_all", model_epoch=649, mri_types=["MR_Flair"], architecture=seganio, dataset=brats2015, dataset_split='merged')

In [ ]:
transfer2019_t1_on_flair2019 = run_evaluation("Transfer_Brats2019_Flair_to_t1", model_epoch=144, mri_types=["flair"], tracked_metric='dice_score_0', architecture=seganio, dataset=brats2019, dataset_split='validation')
transfer2019_t1_freeze_on_flair2019 = run_evaluation("Transfer_Brats2019_Flair_to_t1_freeze_all", model_epoch=1, mri_types=["flair"], tracked_metric='dice_score_0', architecture=seganio, dataset=brats2019, dataset_split='validation')
transfer2019_t1c_freeze_on_flair2019 = run_evaluation("Transfer_Brats2019_Flair_to_t1ce_freeze_all", model_epoch=300, mri_types=["flair"], tracked_metric='dice_score_0', architecture=seganio, dataset=brats2019, dataset_split='validation')
transfer2019_t2_freeze_on_flair2019 = run_evaluation("Transfer_Brats2019_Flair_to_t2_freeze_all", model_epoch=191, mri_types=["flair"], tracked_metric='dice_score_0', architecture=seganio, dataset=brats2019, dataset_split='validation')

In [ ]:
pd.concat([transfer2019_t1_on_flair2019, transfer2019_t1_freeze_on_flair2019, transfer2019_t1c_freeze_on_flair2019, transfer2019_t2_freeze_on_flair2019])

In [ ]:
segan_orig_flair_2015 = run_evaluation("Segan_TF2_brats_on_FLAIR", model_epoch=251, mri_types=["MR_Flair"], architecture=segan_original, tracked_metric='dice_score_0', dataset=brats2015, dataset_split='merged')
segan_orig_flair_on_t2_2015 = run_evaluation("Transfer_segan_original_Brats2015_Flair_to_MR_T2", model_epoch=268, mri_types=["MR_T2"], architecture=segan_original, tracked_metric='dice_score_0', dataset=brats2015, dataset_split='merged')
segan_orig_flair_on_t2_2015_freeze = run_evaluation("Transfer_segan_original_Brats2015_Flair_to_MR_T2_freeze_all", model_epoch=234, mri_types=["MR_T2"], architecture=segan_original, tracked_metric='dice_score_0', dataset=brats2015, dataset_split='merged')
pd.concat([segan_orig_flair_2015, segan_orig_flair_on_t2_2015, segan_orig_flair_on_t2_2015_freeze])

# Graph Generation

In [ ]:
# Melting data to have all the evaluation metrics as new rows
result_to_plot = full_results.melt(id_vars=['model', 'run', 'Source Modality', 'Target Modality', 'Fine Tuning'], var_name='Metric')
# Filtering only meaningful data
result_to_plot = result_to_plot[result_to_plot['Metric'].isin(['dice_score', 'precision', 'sensitivity'])]
result_to_plot = result_to_plot[result_to_plot['run']=='testing']
result_to_plot

In [ ]:
plt.figure(figsize=(16, 8))
# Filter only the Transfer models
temp_data = result_to_plot[result_to_plot['Source Modality'] != 'ALL']
temp_data = temp_data[temp_data['Target Modality'] != 'FLAIR']
#sb.barplot(data=temp_data, y='value', x='Target Modality', hue='Fine Tuning')
# Put the legend out of the figure
sb.catplot(data=temp_data, y='value', col='Metric', hue='Fine Tuning', x='Target Modality', kind='bar', col_wrap=2, sharex=False)

In [ ]:
full_results[full_results['run']=='testing']